# Importing experimental Jun lab data

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Imports

In [2]:
import os, copy, pickle
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

import sys
sys.path.append('code')
from functions_response import process_fsglt, process_fsglt20, fit_normal_fsglt, fit_lognormal_fsglt, process_jts, make_daughterID_jts, make_integer_IDs

sys.path.append(os.path.join('..','colicycle','colicycle'))
import exp_parameters as ep

## Parameters

In [3]:
direxp = os.path.join('.','experiments')
if not os.path.isdir(direxp):
    os.makedirs(direxp)

dirdata = os.path.join('data')

## Load data from Si & Le Treut (2019)

In [4]:
fname = '2019FSGLT.xlsx'
fpath_in = os.path.join(dirdata, fname)

exp_map = {
    'mg1655_acetate': 'MG1655 M9 acetate', \
    'mg1655_glucose': 'MG1655 MOPS glucose', \
    'mg1655_glycerol11aa': 'MG1655 MOPS glycerol 11aa', \
    'ncm3722_arginine': 'NCM3722 MOPS arginine', \
    'ncm3722_glucose': 'NCM3722 MOPS glucose', \
    'ncm3722_glucose12aa': 'NCM3722 MOPS glucose 12aa' \
}

for name in exp_map.keys():
    sheet_name = exp_map[name]
    print("Importing experiment {:s}".format(sheet_name))
    df = pd.read_excel(fpath_in, sheet_name=sheet_name)
    
    sel = ['elongation rate (1/hour)',
           'initiation size per ori (micron)',
           'division size (micron)', 'newborn size (micron)',
           'added size (micron)', 'cell width (micron)', 
           'generation time (minute)', 'tau_cyc (minute)', 'septum position',
           'cell ID', 'daughter ID']

    # reorder columns
    df = df[sel]

    # rename columns
    col_mapping = {
        'elongation rate (1/hour)': 'lambda', \
        'initiation size per ori (micron)': 'Lambda_i', \
        'tau_cyc (minute)': 'taucyc', \
        'division size (micron)': 'Sd', \
        'newborn size (micron)': 'Sb', \
        'added size (micron)': 'Delta_bd', \
        'cell width (micron)': 'width', \
        'generation time (minute)': 'tau', \
        'tau_cyc (minute)': 'tau_cyc', \
        'septum position': 'phi'}

    df.rename(columns=col_mapping, inplace=True)

    # convert growth rate to per minutes
    df['lambda'] = df['lambda'] / 60.
    df['tau_eff'] = np.log(2.)/df['lambda']

    # process the data frame to add attributes
    process_fsglt(df)
    
    # choose a method for the computation of delta_id and delta_ii
    # see source code for details
    col_mapping = { \
                   'delta_id_m1': 'delta_id', \
                   'delta_ii_forward': 'delta_ii', \
                   }
    df.rename(columns=col_mapping, inplace=True)

    # save the dataframe
    fname = "{:s}.pkl".format(name)
    outputdir = os.path.join(direxp, name)
    if not os.path.isdir(outputdir):
        os.makedirs(outputdir)
    fpath = os.path.join(outputdir,'colidata.pkl')
    df.to_pickle(fpath)
    print(fpath)

Importing experiment MG1655 M9 acetate
./experiments/mg1655_acetate/colidata.pkl
Importing experiment MG1655 MOPS glucose
./experiments/mg1655_glucose/colidata.pkl
Importing experiment MG1655 MOPS glycerol 11aa
./experiments/mg1655_glycerol11aa/colidata.pkl
Importing experiment NCM3722 MOPS arginine
./experiments/ncm3722_arginine/colidata.pkl
Importing experiment NCM3722 MOPS glucose
./experiments/ncm3722_glucose/colidata.pkl
Importing experiment NCM3722 MOPS glucose 12aa
./experiments/ncm3722_glucose12aa/colidata.pkl


## Load data from Le Treut & Si & Li (2020) -- MG1655 data

In [5]:
fname = '2020GLTFSDL.xlsx'
fpath_in = os.path.join(dirdata, fname)

exp_map = {
#     'bw25113_glucose': 'BW25113 MOPS glucose', \
#     'bw25113_glucose_uracil': 'BW25113 MOPS glucose uracil', \
#     'jh642_mannose': 'B. subtilis S750 mannose', \
    'mg1655_acetate_uracil': 'MG1655 MOPS acetate uracil' , \
    'mg1655_glycerol6aa_uracil': 'MG1655 MOPS glycerol 6aa uracil', \
    'mg1655_glucose_uracil': 'MG1655 MOPS glucose uracil', \
}

for name in exp_map.keys():
    sheet_name = exp_map[name]
    print("Importing experiment {:s}".format(sheet_name))
    df = pd.read_excel(fpath_in, sheet_name=sheet_name)
    
    sel = ['elongation rate (1/hour)',
           'initiation size per ori (micron)',
           'division size (micron)', 'newborn size (micron)',
           'added size (micron)', 'cell width (micron)', 
           'generation time (minute)', 'tau_cyc (minute)', 'septum position',
           'cell ID', 'daughter ID']

    # reorder columns
    df = df[sel]

    # rename columns
    col_mapping = {
        'elongation rate (1/hour)': 'lambda', \
        'initiation size per ori (micron)': 'Lambda_i', \
        'tau_cyc (minute)': 'taucyc', \
        'division size (micron)': 'Sd', \
        'newborn size (micron)': 'Sb', \
        'added size (micron)': 'Delta_bd', \
        'cell width (micron)': 'width', \
        'generation time (minute)': 'tau', \
        'tau_cyc (minute)': 'tau_cyc', \
        'septum position': 'phi'}

    df.rename(columns=col_mapping, inplace=True)

    # convert growth rate to per minutes
    df['lambda'] = df['lambda'] / 60.
    df['tau_eff'] = np.log(2.)/df['lambda']

    # process the data frame to add attributes
    process_fsglt(df)
    
    # choose a method for the computation of delta_id and delta_ii
    # see source code for details
    col_mapping = { \
                   'delta_id_m1': 'delta_id', \
                   'delta_ii_forward': 'delta_ii', \
                   }
    df.rename(columns=col_mapping, inplace=True)

    # save the dataframe
    fname = "{:s}.pkl".format(name)
    outputdir = os.path.join(direxp, name)
    if not os.path.isdir(outputdir):
        os.makedirs(outputdir)
    fpath = os.path.join(outputdir,'colidata.pkl')
    df.to_pickle(fpath)
    print(fpath)

Importing experiment MG1655 MOPS acetate uracil
./experiments/mg1655_acetate_uracil/colidata.pkl
Importing experiment MG1655 MOPS glycerol 6aa uracil
./experiments/mg1655_glycerol6aa_uracil/colidata.pkl
Importing experiment MG1655 MOPS glucose uracil
./experiments/mg1655_glucose_uracil/colidata.pkl


## Load data from Le Treut & Si & Li (2020) -- BW and B. subtilis data
In this dataformat, there is not cell ID nor daughter ID information. The initiation-to-initiation adder was directly computed in upstream data processing steps.

In [6]:
fname = '2020GLTFSDL.xlsx'
fpath_in = os.path.join(dirdata, fname)

exp_map = {
    'bw25113_glucose': 'BW25113 MOPS glucose', \
    'bw25113_glucose_uracil': 'BW25113 MOPS glucose uracil', \
    'jh642_mannose': 'B. subtilis S750 mannose'
}

for name in exp_map.keys():
    sheet_name = exp_map[name]
    print("Importing experiment {:s}".format(sheet_name))
    df = pd.read_excel(fpath_in, sheet_name=sheet_name)
    
    sel = ['elongation rate (1/hour)',
           'initiation size per ori (micron)',
           'division size (micron)', 'newborn size (micron)',
           'added size (micron)', 'cell width (micron)', 
           'generation time (minute)', 'tau_cyc (minute)', 'septum position',
           'added initiation size per ori (micron)']

    # reorder columns
    df = df[sel]

    # rename columns
    col_mapping = {
        'elongation rate (1/hour)': 'lambda', \
        'initiation size per ori (micron)': 'Lambda_i', \
        'tau_cyc (minute)': 'taucyc', \
        'division size (micron)': 'Sd', \
        'newborn size (micron)': 'Sb', \
        'added size (micron)': 'Delta_bd', \
        'added initiation size per ori (micron)': 'delta_ii_forward', \
        'cell width (micron)': 'width', \
        'generation time (minute)': 'tau', \
        'tau_cyc (minute)': 'tau_cyc', \
        'septum position': 'phi'}

    df.rename(columns=col_mapping, inplace=True)

    # convert growth rate to per minutes
    df['lambda'] = df['lambda'] / 60.
    df['tau_eff'] = np.log(2.)/df['lambda']

    # process the data frame to add attributes
    process_fsglt20(df)

    # choose a method for the computation of delta_id and delta_ii
    # see source code for details
    col_mapping = { \
                   'delta_id_m1': 'delta_id', \
                   'delta_ii_forward': 'delta_ii', \
                   }
    df.rename(columns=col_mapping, inplace=True)

    # save the dataframe
    fname = "{:s}.pkl".format(name)
    outputdir = os.path.join(direxp, name)
    if not os.path.isdir(outputdir):
        os.makedirs(outputdir)
    fpath = os.path.join(outputdir,'colidata.pkl')
    df.to_pickle(fpath)
    print(fpath)

Importing experiment BW25113 MOPS glucose
./experiments/bw25113_glucose/colidata.pkl
Importing experiment BW25113 MOPS glucose uracil
./experiments/bw25113_glucose_uracil/colidata.pkl
Importing experiment B. subtilis S750 mannose
./experiments/jh642_mannose/colidata.pkl


## Load data from Sauls et al (2020)

In [7]:
fname = '2020JTS.xlsx'
fpath_in = os.path.join(dirdata, fname)

exp_map = {
    '3610_gly': 'gly+ 0 uM cam', \
    '3610_man': 'man 0 uM cam', \
    '3610_suc': 'suc 0 uM cam', \
}


for name in exp_map.keys():
    sheet_name = exp_map[name]
    print("Importing experiment {:s}".format(sheet_name))
    df = pd.read_excel(fpath_in, sheet_name=sheet_name)
    
    # re-name column in agreement with Si & Le Treut (2019) column names
    df.reset_index(inplace=True)
    col_mapping = {
    'id': 'cell ID', \
    'birth length (micron)': 'newborn size (micron)', \
    'division length (micron)': 'division size (micron)', \
    'width (micron)': 'cell width (micron)', \
    'added division length (micron)': 'added size (micron)', \
    'growth rate (1/hour)': 'elongation rate (1/hour)', \
    'C+D (minute)': 'tau_cyc (minute)', \
    'initiation length per ori (micron)': 'initiation size per ori (micron)'
    }

    df.rename(columns=col_mapping, inplace=True)

    # replace None string by None value
    for key in df.columns:
        idx = df[key].eq('None')
        df.loc[idx, key] = None
    
    # create integer index
    ncells = len(df)
    df['index'] = range(ncells)

    # create daugther ID column and keep only mother cells
    make_daughterID_jts(df)
    make_integer_IDs(df)
    

    # fall back on previous formatting
    sel = ['elongation rate (1/hour)',
           'initiation size per ori (micron)',
           'division size (micron)', 'newborn size (micron)',
           'added size (micron)', 'cell width (micron)', 
           'generation time (minute)', 'tau_cyc (minute)', 'septum position',
           'cell ID', 'daughter ID']

    # reorder columns
    df = df[sel]

    # rename columns
    col_mapping = {
        'elongation rate (1/hour)': 'lambda', \
        'initiation size per ori (micron)': 'Lambda_i', \
        'tau_cyc (minute)': 'taucyc', \
        'division size (micron)': 'Sd', \
        'newborn size (micron)': 'Sb', \
        'added size (micron)': 'Delta_bd', \
        'cell width (micron)': 'width', \
        'generation time (minute)': 'tau', \
        'tau_cyc (minute)': 'tau_cyc', \
        'septum position': 'phi'}

    df.rename(columns=col_mapping, inplace=True)

    # convert growth rate to per minutes
    df['lambda'] = df['lambda'] / 60.
    df['tau_eff'] = np.log(2.)/df['lambda']

    # process the data frame to add attributes
    process_jts(df)  # add-hoc processing
    
    # choose a method for the computation of delta_id and delta_ii
    # see source code for details
    col_mapping = { \
                   'delta_id_m1': 'delta_id', \
                   'delta_ii_forward': 'delta_ii', \
                   }
    df.rename(columns=col_mapping, inplace=True)

    # save the dataframe
    fname = "{:s}.pkl".format(name)
    outputdir = os.path.join(direxp, name)
    if not os.path.isdir(outputdir):
        os.makedirs(outputdir)
    fpath = os.path.join(outputdir,'colidata.pkl')
    df.to_pickle(fpath)
    print(fpath)

Importing experiment gly+ 0 uM cam


/usr/local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


skipping cell f16p0685t0472r01 with daughters f16p0685t0491r02 and f16p0685t0491r03
skipping cell f31p0255t0423r01 with daughters f31p0255t0453r02 and f31p0255t0453r03
./experiments/3610_gly/colidata.pkl
Importing experiment man 0 uM cam
skipping cell f03p0256t0247r01 with daughters f03p0256t0275r02 and f03p0256t0275r03
skipping cell f07p0298t0406r01 with daughters f07p0298t0446r02 and f07p0298t0446r03
skipping cell f30p0430t0262r01 with daughters f30p0430t0288r02 and f30p0430t0288r03
skipping cell f33p0518t0146r01 with daughters f33p0518t0173r02 and f33p0518t0173r03
./experiments/3610_man/colidata.pkl
Importing experiment suc 0 uM cam
skipping cell f20p0516t0437r01 with daughters f20p0516t0473r02 and f20p0516t0473r03
./experiments/3610_suc/colidata.pkl


## Save parameters for simulations

In [8]:
col_mapping = {
    'tau_eff': 'tau_fit', \
    'Sb': 'Lb_fit', \
    'Sd': 'Ld_fit', \
    'Lambda_i': 'Li_fit', \
    'delta_ii': 'DLi', \
    'delta_id': 'DLdLi', \
    'Delta_bd': 'dL', \
    'phi': 'phi', \
    'mother ID': 'mother_id',\
    'cell ID': 'cell_id'}

In [9]:
# the dataset without lineage information will produce an error for the calculation
# of the mother/daughter correlation for the growth rate.
names = [
    'mg1655_acetate', \
    'mg1655_glucose', \
    'mg1655_glycerol11aa', \
    'ncm3722_arginine', \
    'ncm3722_glucose', \
    'ncm3722_glucose12aa', \
#     'bw25113_glucose', \
#     'bw25113_glucose_uracil', \
    'mg1655_acetate_uracil' , \
    'mg1655_glycerol6aa_uracil', \
    'mg1655_glucose_uracil', \
#     'jh642_mannose', \
    '3610_gly', \
    '3610_man', \
    '3610_suc', \
]

for name in names:
    inputdir = os.path.join(direxp, name)
    fname = "{:s}.pkl".format('colidata')
    fpath = os.path.join(inputdir,fname)
    colidata = pd.read_pickle(fpath)
    print("Loaded {:s}".format(name))
    
    # rename and select useful columns
    sel = list(col_mapping.values())
    colidata = colidata.rename(columns=col_mapping)
    colidata = colidata[sel]
    colidata=colidata.set_index('cell_id')
    colidata
    
    # compute parameters
    
    tau_corr = ep.calculate_tau_correlation(colidata,'tau_fit')
    bin_pos_tau, valbins_tau, res_fit_tau = fit_lognormal_fsglt(colidata['tau_fit'].dropna(),np.arange(2,6,0.1))
    bin_pos_DLi, valbins_DLi, res_fit_DLi = fit_normal_fsglt(colidata['DLi'].dropna(),np.arange(0,2,0.1))
    bin_pos_DLdLi, valbins_DLdLi, res_fit_DLdLi = fit_lognormal_fsglt(colidata['DLdLi'].dropna(),np.arange(-1,3,0.1))
    bin_pos_Lb, valbins_Lb, res_fit_Lb = fit_normal_fsglt(colidata['Lb_fit'].dropna(),np.arange(0,7,0.1))
    bin_pos_Lblog, valbins_Lblog, res_fit_Lblog = fit_lognormal_fsglt(colidata['Lb_fit'].dropna(),np.arange(-1,3,0.1))
    bin_pos_dL, valbins_dL, res_fit_dL = fit_normal_fsglt(colidata['dL'].dropna(),np.arange(0,7,0.1))

    # same method won't work because only mother cells are retained here
    # no 2 cells with same mother.
    # divR_std = ep.calculate_div_ratio(colidata)
    phi = colidata['phi'].to_numpy()
    divR = phi / (1.-phi)
    divR = np.concatenate([divR, 1/divR])
    divR_std = np.nanstd(divR)

    # store the parameters
    param_storage={}
    param_storage['tau_corr'] = tau_corr
    param_storage['fit_logtau'] = res_fit_tau.x
    param_storage['fit_DLi'] = res_fit_DLi.x
    param_storage['fit_logDLdLi'] = res_fit_DLdLi.x
    param_storage['fit_Lb'] = res_fit_Lb.x
    param_storage['fit_logLb'] = res_fit_Lblog.x
    param_storage['fit_dL'] = res_fit_dL.x
    param_storage['divR_std'] = divR_std
    
    outputdir = os.path.join(direxp, name)
    fpath = os.path.join(outputdir,'simul_params.pkl')
    with open(fpath, 'wb') as f:
        pickle.dump(param_storage, f, pickle.HIGHEST_PROTOCOL)
        
    # plots
    fig,axes = plt.subplots(2,3, figsize=(15,5))
    axes[0,0].plot(bin_pos_tau, valbins_tau,'o')
    axes[0,0].plot(bin_pos_tau, ep.fun_single_gauss(bin_pos_tau, *res_fit_tau.x))
    axes[0,0].set_title('tau_fit')
    axes[0,1].plot(bin_pos_DLi, valbins_DLi,'o')
    axes[0,1].plot(bin_pos_DLi, ep.fun_single_gauss(bin_pos_DLi, *res_fit_DLi.x))
    axes[0,1].set_title('DLi')
    axes[0,2].plot(bin_pos_DLdLi, valbins_DLdLi,'o')
    axes[0,2].plot(bin_pos_DLdLi, ep.fun_single_gauss(bin_pos_DLdLi, *res_fit_DLdLi.x))
    axes[0,2].set_title('DLdLi')
    axes[1,0].plot(bin_pos_Lb, valbins_Lb,'o')
    axes[1,0].plot(bin_pos_Lb, ep.fun_single_gauss(bin_pos_Lb, *res_fit_Lb.x))
    axes[1,0].set_title('Lb')
    axes[1,1].plot(bin_pos_Lblog, valbins_Lblog,'o')
    axes[1,1].plot(bin_pos_Lblog, ep.fun_single_gauss(bin_pos_Lblog, *res_fit_Lblog.x))
    axes[1,1].set_title('Lb_log')
    axes[1,2].plot(bin_pos_dL, valbins_dL,'o')
    axes[1,2].plot(bin_pos_dL, ep.fun_single_gauss(bin_pos_dL, *res_fit_dL.x))
    axes[1,2].set_title('dL')
    
    outputdir = os.path.join(direxp, name)
    fpath = os.path.join(outputdir,'fit_figure')
    fig.savefig(fpath + '.png', dpi=300, bbox_inches='tight', pad_inches=0 )
    plt.close('all')

Loaded mg1655_acetate
Loaded mg1655_glucose
Loaded mg1655_glycerol11aa
Loaded ncm3722_arginine
Loaded ncm3722_glucose
Loaded ncm3722_glucose12aa
Loaded mg1655_acetate_uracil
Loaded mg1655_glycerol6aa_uracil
Loaded mg1655_glucose_uracil
Loaded 3610_gly
Loaded 3610_man
Loaded 3610_suc
